Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Logistic model

In [5]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + 0.01*tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 49.997528
Minibatch accuracy: 9.4%
Validation accuracy: 13.1%
Minibatch loss at step 500: 0.992192
Minibatch accuracy: 78.1%
Validation accuracy: 81.0%
Minibatch loss at step 1000: 0.690550
Minibatch accuracy: 84.4%
Validation accuracy: 80.4%
Minibatch loss at step 1500: 0.593720
Minibatch accuracy: 85.9%
Validation accuracy: 80.9%
Minibatch loss at step 2000: 0.591715
Minibatch accuracy: 84.4%
Validation accuracy: 80.2%
Minibatch loss at step 2500: 0.813985
Minibatch accuracy: 78.1%
Validation accuracy: 78.0%
Minibatch loss at step 3000: 0.842189
Minibatch accuracy: 79.7%
Validation accuracy: 80.6%
Test accuracy: 87.7%


### Neural network model

In [7]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    hidden_layer_size = 1024
    
    
    weights1 = tf.Variable(
        tf.truncated_normal([image_size*image_size, hidden_layer_size]))
    biases1 = tf.Variable(tf.zeros([hidden_layer_size]))
    
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    
    weights2 = tf.Variable(
    tf.truncated_normal([hidden_layer_size, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(hidden, weights2) + biases2
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + 0.005*tf.nn.l2_loss(weights1) + 0.005*tf.nn.l2_loss(weights2)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1)+biases1), weights2) + biases2
    )
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1)+biases1), weights2) + biases2
    )
    
    

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step", step, ":", l)
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0 : 1916.97
Minibatch accuracy: 7.0%
Validation accuracy: 25.1%
Test accuracy: 26.7%
Minibatch loss at step 500 : 126.735
Minibatch accuracy: 80.5%
Validation accuracy: 82.0%
Test accuracy: 89.3%
Minibatch loss at step 1000 : 10.8542
Minibatch accuracy: 86.7%
Validation accuracy: 84.0%
Test accuracy: 91.1%
Minibatch loss at step 1500 : 1.35738
Minibatch accuracy: 89.1%
Validation accuracy: 84.6%
Test accuracy: 91.0%
Minibatch loss at step 2000 : 0.602323
Minibatch accuracy: 88.3%
Validation accuracy: 84.8%
Test accuracy: 91.4%
Minibatch loss at step 2500 : 0.637556
Minibatch accuracy: 83.6%
Validation accuracy: 84.5%
Test accuracy: 91.2%
Minibatch loss at step 3000 : 0.699925
Minibatch accuracy: 82.8%
Validation accuracy: 85.2%
Test accuracy: 91.9%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (128*3)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step", step, ":", l)
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0 : 1871.86
Minibatch accuracy: 19.5%
Validation accuracy: 30.5%
Test accuracy: 33.2%
Minibatch loss at step 500 : 128.325
Minibatch accuracy: 100.0%
Validation accuracy: 72.2%
Test accuracy: 80.0%
Minibatch loss at step 1000 : 10.5868
Minibatch accuracy: 100.0%
Validation accuracy: 76.3%
Test accuracy: 83.6%
Minibatch loss at step 1500 : 1.03424
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Test accuracy: 85.0%
Minibatch loss at step 2000 : 0.244935
Minibatch accuracy: 100.0%
Validation accuracy: 77.7%
Test accuracy: 84.9%
Minibatch loss at step 2500 : 0.174727
Minibatch accuracy: 100.0%
Validation accuracy: 77.9%
Test accuracy: 85.0%
Minibatch loss at step 3000 : 0.160499
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Test accuracy: 85.0%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [10]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    hidden_layer_size = 1024
    
    
    weights1 = tf.Variable(
        tf.truncated_normal([image_size*image_size, hidden_layer_size]))
    biases1 = tf.Variable(tf.zeros([hidden_layer_size]))
    
    hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1), 0.5)
    
    weights2 = tf.Variable(
    tf.truncated_normal([hidden_layer_size, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(hidden, weights2) + biases2
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + 0.005*tf.nn.l2_loss(weights1) + 0.005*tf.nn.l2_loss(weights2)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1)+biases1), weights2) + biases2
    )
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1)+biases1), weights2) + biases2
    )
    
    

In [11]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (128*3)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step", step, ":", l)
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0 : 2074.27
Minibatch accuracy: 17.2%
Validation accuracy: 23.2%
Test accuracy: 24.9%
Minibatch loss at step 500 : 129.105
Minibatch accuracy: 100.0%
Validation accuracy: 76.3%
Test accuracy: 84.3%
Minibatch loss at step 1000 : 10.6263
Minibatch accuracy: 100.0%
Validation accuracy: 77.4%
Test accuracy: 85.2%
Minibatch loss at step 1500 : 1.04877
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Test accuracy: 85.2%
Minibatch loss at step 2000 : 0.259875
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Test accuracy: 85.2%
Minibatch loss at step 2500 : 0.186658
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Test accuracy: 85.3%
Minibatch loss at step 3000 : 0.174508
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Test accuracy: 85.4%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [15]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    hidden_layer_size1 = 1024
    
    
    weights1 = tf.Variable(
        tf.truncated_normal([image_size*image_size, hidden_layer_size1]))
    biases1 = tf.Variable(tf.zeros([hidden_layer_size1]))
    
    hidden1 = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1), 0.8)
    
    hidden_layer_size2 = 300
    weights2 = tf.Variable(
        tf.truncated_normal([hidden_layer_size1, hidden_layer_size2]))
    biases2 = tf.Variable(tf.zeros([hidden_layer_size2]))
    
    hidden2 = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden1, weights2) + biases2), 0.8)
    
    hidden_layer_size3 = 50
    weights3 = tf.Variable(
        tf.truncated_normal([hidden_layer_size2, hidden_layer_size3]))
    biases3 = tf.Variable(tf.zeros([hidden_layer_size3]))
    
    hidden3 = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden2, weights3) + biases3), 0.8)
    
    weights4 = tf.Variable(
    tf.truncated_normal([hidden_layer_size3, num_labels]))
    biases4 = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(hidden3, weights4) + biases4
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + 0.0025*tf.nn.l2_loss(weights1) + 0.0025*tf.nn.l2_loss(weights2) + 0.0025*tf.nn.l2_loss(weights3) + 0.0025*tf.nn.l2_loss(weights4)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2), weights3) + biases3), weights4) + biases4
    )
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2), weights3) + biases3), weights4) + biases4
    )
    
    

In [16]:
num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step", step, ":", l)
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      #print("Validation accuracy: %.1f%%" % accuracy(
        #valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0 : 2.44418e+11
Minibatch accuracy: 10.2%
Test accuracy: 10.0%
Minibatch loss at step 500 : nan
Minibatch accuracy: 6.2%
Test accuracy: 10.0%
Minibatch loss at step 1000 : nan
Minibatch accuracy: 8.6%
Test accuracy: 10.0%


KeyboardInterrupt: 